In [ ]:
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from tqdm import tqdm_notebook as tqdmn
import warnings

warnings.filterwarnings("ignore")

#Path to geckodriver.exe
exec_path = 'geckodriver.exe'
#Path to firefox.exe
binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'

def scrape_coordinates(fileinput, sheetname, address_col):
    #Read input file
    try:
        if file.rsplit('.', 1)[-1] == 'xlsx':
            df = pd.read_excel(fileinput, sheet_name= sheetname)
        elif file.rsplit('.', 1)[-1] == 'csv':
            df = pd.read_csv(fileinput)

    except:
        raise Exception("File input error")
    
    #Setup Scraper
    options = Options()
    options.set_headless(headless=True)
    options.binary = binary_path
    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = True

    driver = webdriver.Firefox(firefox_options=options, capabilities=cap, executable_path= exec_path)
    
    #Scraping coordinates
    url_with_coordinates = []
    lat_scrape = []
    long_scrape = []
    colnames = address_col
    for item in tqdmn(df[colnames]):
        url = 'https://www.google.com/maps/search/' + str(item)
        driver.get(url)
        url_with_coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
    driver.close()
    
    df['url_with_coordinates'] = url_with_coordinates
    for url in df['url_with_coordinates']:
        try:
            lat_scrape.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0])
        except:
            lat_scrape.append('NA')
        try:
            long_scrape.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1])
        except:
            long_scrape.append('NA')
            
    df['lat_scrape'] = lat_scrape
    df['long_scrape'] = long_scrape
    
    return df

fileinput = 'YOUR FILE'
sheetname = 'SHEET NAME'
address_col = 'CORRESPONDING ADDRESS COLUMN'

df_out = scrape_coordinates(fileinput, sheetname, address_col)
df_out.to_excel('scraper_output.xlsx', index = False)